# Introduction

Queen Mary University of London, School of Electronic Engineering and Computer Science.

**Author**: original notebook by Raluca Gaina, updates by Simon Colton

ECS7022P: Computational Creativity
===

---

# **Practical 3. Implementing Neural Style Transfer**

In this practical you will implement a method to take a *content image* and activate it to gain some of the visual look of a *style image*. To do this, we'll think of the RGB values in the image as being trainable, and define a loss function which drives the training. The loss function will be based on a pre-trained machine vision model that you will download. There will be two terms to the loss function as per lecture 3: one term using the output of the early layers of the pre-trained model (to gain the fine detail of the style image) and another term using the output of the final layer of the pre-trained model (to preserve the content of the content image).

Please turn on "Menu" => "View => "Table of Contents" to navigate this notebook.


## TODO 1 - Copy the Notebook and Enable GPU Runtime

Create your own copy of this Colab notebook, using "Menu" => "File" => "Save a Copy In Drive" and then opening that copy. You should now be able to edit that version and save your changes.

Make sure that you enable GPU for the session, using "Menu" => "Runtime" => "Change Runtime Type".

**Important**:  
When you have finished the practical, make sure you delete and disconnect the runtime, otherwise you may have restricted GPU access next time you use Colab notebooks.


# Setting Up

In [ ]:
#@title TODO 2 - Imports and Checking GPUs

#@markdown View the code in this cell to see which imports are required for the
#@markdown notebook. We are using TensorFlow, a well known deep learning development
#@markdown kit. See how we've used tensorflow to check whether a GPU has been assigned.
#@markdown If it hasn't found one, try connecting to a GPU via the runtime menu,
#@markdown and then running this cell again. To keep things tidy, once run, clear
#@markdown the output from this cell

#@markdown Search online for the tensorflow.keras.applications package - what
#@markdown other pre-trained models does it have available?

import tensorflow as tf
import keras
import numpy as np
from tensorflow.keras.utils import get_file
import matplotlib.pyplot as plt
from tensorflow.keras import applications
from tensorflow.keras.utils import plot_model
from keras import Model
from tensorflow.keras.optimizers import SGD
from IPython import display
import os

device_name = tf.test.gpu_device_name()
print('Found GPU at: {}'.format(device_name))
print("TensorFlow version:", tf.__version__)

Found GPU at: /device:GPU:0
TensorFlow version: 2.15.0


## TODO 3 - Downloading the Machine Vision Model

In this practical, we're using a pre-trained model, as is typical for such applications. Therefore, for our network definition we simply load a suitable model. The model used has been shown in [some tests](https://towardsdatascience.com/practical-techniques-for-getting-style-transfer-to-work-19884a0d69eb) to influence the resulting images, so this is one point to experiment with when looking for particular outcomes.

Here, we'll use the [VGG19 model](https://iq.opengenus.org/vgg19-architecture/#:~:text=VGG19%20is%20a%20variant%20of,VGG19%20has%2019.6%20billion%20FLOPs.) provided with Keras. VGG16 is also an option compatible with the notebook - other models would require some more adjustments in setting up the model and image preprocessing.

Look at the code in the cell below to see how we use Keras applications to download the pre-trained model.

---

**Question**: Why is it called VGG19?

**Answer**:

VGG = (V)isual (G)eometry (G)roup (at the University of Oxford)

19 refers to the version of VGG having 19 layers (16 convolutional + 3 fully connected).

In [ ]:
network_choice = "vgg19" #@param['vgg19', 'vgg16']

# Note that there is a network object and a model object (they're different!)

network_dict = {
    "vgg19": applications.vgg19,
    "vgg16": applications.vgg16,
}
network = network_dict[network_choice]

model_dict = {
    "vgg19": applications.vgg19.VGG19(weights="imagenet", include_top=False),
    "vgg16": applications.vgg16.VGG16(weights="imagenet", include_top=False),
}

# We will call this later when needed
def build_model():
    model = model_dict[network_choice]
    model.summary()
    return model


58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
#@title ## TODO 4 - Set up a Function to View Images

#@markdown There is one last thing to set up, which is to define a function which can
#@markdown be used to view our content, style and style-transferred image.
#@markdown Currently, this function is not finished.
#@markdown Cut and paste the relevant code to finish this function off, by enabling it
#@markdown to show the style transfer image as well as the content and style image


# Displays content, style and combination images, labelled. Any could be missing if set to None
def display_images(content_image=None, style_image=None, style_transfer_image=None):
    # For each image, displaying in the subplot if not `None`
    _, cells = plt.subplots(1, 3, figsize=(45,15))
    if content_image is not None:
        cells[0].imshow(content_image)
        cells[0].set_title("Content", fontsize=30)
    if style_image is not None:
        cells[1].imshow(style_image)
        cells[1].set_title("Style", fontsize=30)
    # TODO 4.1 add style transfer image to the cells
    if style_transfer_image is not None:
        cells[2].imshow(style_transfer_image)
        cells[2].set_title("Style transfer", fontsize=30)
    for cell in cells:
        cell.axis('off')
    plt.show()

# Loading and Preparing the Image Data

In [ ]:
#@title # TODO 5 - Download Images
#@markdown Add code to this cell which uses the `keras.utils.get_file` and `load_img` function to download these
#@markdown images and load them into keras images with variable names `style_image` and `content_image`.

#@markdown Your code should also use keras.utils.load_img to load the image.
#@markdown Choose other image URLs if you have things you want to try out!

style_image_url = "https://i.imgur.com/9ooB60I.jpg" # @param {type: "string"}
content_image_url = "https://tourism.euskadi.eus/contenidos/d_destinos_turisticos/0000004981_d2_rec_turismo/en_4981/images/CT_cabecerabilbaoguggen.jpg" # @param {type: "string"}

# TODO 5.1
style_image_path = keras.utils.get_file(origin=style_image_url)
content_image_path = keras.utils.get_file(origin=content_image_url)

# TODO 5.2
style_image = keras.utils.load_img(style_image_path)
content_image = keras.utils.load_img(content_image_path)

201498/201498 [==============================] - 2s 10us/step


In [ ]:
#@title # TODO 6 - Use this cell to view the images you have downloaded

#@markdown Click the images to see them in detail.

display_images(content_image, style_image)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
#@title # TODO 7 - Extract and Scale the Image Data

#@markdown Look in this cell to see how the image data is extracted.
#@markdown Choose a height (img_height) for your stylised image

# Set the img_height variable
img_height = 400 #@param{type: 'number'}

# Turn images into Numpy arrays to find dimensions
content_image_np = keras.utils.img_to_array(content_image)
style_image_np = keras.utils.img_to_array(style_image)

print(f"The content image shape is: {content_image_np.shape}")
print(f"The style image shape is: {style_image_np.shape}")

# Setting image dimensions variables
height = content_image_np.shape[0]
width = content_image_np.shape[1]
channels = content_image_np.shape[2]

# Scale the width appropriately to get the img_width variable
img_width = int(width * (img_height/height))

# Set the img_size variable (used later)
img_size = img_height * img_width

The content image shape is: (1200, 1920, 3)
The style image shape is: (934, 1160, 3)


# Implementing Image Handling Functions

We need to preprocess the images to be compatible with the expected input for the network used. In our case (VGG19/VGG16 models), we need a tensor representing a batch of images (even if batch size is 1 when processing images individually). Importantly, images also need to be pre-processed by the corresponding model using the Keras `preprocess_input(img)` function.

As the network outputs tensors, we'll also need a utility function to turn tensors back into Numpy array images in order to display the results. To reverse the `preprocess_input(img)` function, we add the mean values for each channel of the Imagenet dataset (found [here](https://github.com/DeepVoltaire/AutoAugment/issues/4)) and transform the image from BGR to RGB.

Examine the code below to see how we process an image for the model, and then translate the output back to an image.

In [ ]:
#@title # TODO 8 - Answer Questions in the Code
# Open, resize and format pictures into appropriate tensors
def preprocess_image(image_path):
    # Load image with given size into PIL format
    img = keras.utils.load_img(image_path, target_size=(img_height,img_width))

    # Turn image into Numpy array
    img = keras.utils.img_to_array(img)

    # The next step expects a batch of images, so add another dimension
    img = np.expand_dims(img, axis=0)

    # Call the network preprocessing step.
    # This converts to BGR and zero-centers data with regards to the Imagenet dataset,
    # by subtracting the mean channel values in Imagenet
    img = network.preprocess_input(img)

    # Transform the img to a tensor and return this
    return tf.convert_to_tensor(img)


# Reverse the preprocessing step to turn the tensor into an RGB Numpy array which we can visualise
def deprocess_image(tensor):
    # Transform the tensor into a Numpy array
    img = tensor.numpy()[0]

    # Return BGR values to normal (non-zero-centered), adding back in the means of the Imagenet dataset

    # TODO 8.1 - what does the [:, :, 0] notation below mean?
    '''
    Answer:
    [:, :, 0] refers to the 1st element (i.e. at index 0) of each 3rd
    dimensional array, i.e. we select all 1st dimensional arrays, all 2nd
    dimensional arrays and the 1st index's value of the 3rd dimensional arrays.
    For example, [:, :, 0] for the array...
    [[[1, 2], [3, 4]], [[5, 6], [7, 8]]]
    ... becomes...
    [[[1], [3]], [[5], [7]]]

    NOTE:
    In our case, the 3rd dimension consists of arrays size 3 and refers to:
    - R (red, at index = 0)
    - G (green, at index = 1)
    - B (blue, at index = 2)
    '''
    img[:, :, 0] += 103.939
    img[:, :, 1] += 116.779
    img[:, :, 2] += 123.68

    # Convert from BGR to RGB
    # TODO 8.2 - what does the [:, :, ::-1] notation mean?
    '''
    Answer:
    [:, :, ::-1] reverses the order of the elements in the 3rd dimension, i.e.
    every 3rd dimensional array becomes reversed (each, not together).
    For example...
    [[[1, 2], [3, 4]], [[5, 6], [7, 8]]]
    ... becomes...
    [[[2, 1], [4, 3]], [[6, 5], [8, 7]]]
    '''
    img = img[:, :, ::-1]

    # Ensure values are in valid ranges
    # TODO 8.3 - what does the clip function do?
    '''
    Answer:
    `numpy.clip` clips, i.e. limits the values in an array. Given an interval,
    values outside the interval are clipped to the interval edges. For example,
    if an interval of [0, 1] is specified, values smaller than 0 become 0, and
    values larger than 1 become 1.

    SIDE NOTE:
    Equivalent to but faster than `np.minimum(a_max, np.maximum(a, a_min))`,
    given a xlipping, i.e. limiting interval [a_min, a_max].

    REFERENCE:
    https://numpy.org/doc/stable/reference/generated/numpy.clip.html
    '''

    img = np.clip(img, 0, 255).astype("uint8")
    # Here, we are limiting the values to between 0 and 255 (inclusive).

    return img

**DEMO**: A small demo of the reversing technique used above:

In [ ]:
a = np.array([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
print("Unreversed:\n", a)
print("\nReversed in 1st dimension:\n", a[::-1, :, :])
print("\nReversed in 2nd dimension:\n", a[:, ::-1, :])
print("\nReversed in 3rd dimension:\n", a[:, :, ::-1])

Unreversed:
 [[[1 2]
  [3 4]]

 [[5 6]
  [7 8]]]

Reversed in 1st dimension:
 [[[5 6]
  [7 8]]

 [[1 2]
  [3 4]]]

Reversed in 2nd dimension:
 [[[3 4]
  [1 2]]

 [[7 8]
  [5 6]]]

Reversed in 3rd dimension:
 [[[2 1]
  [4 3]]

 [[6 5]
  [8 7]]]


In [ ]:
#@title # TODO 9 - Implement Code to test the Encoding/Decoding of Images

#@markdown Fill out the code in this cell to check that your pre_processing and deprocessing
#@markdown functions work.

tensor = preprocess_image(style_image_path)
print(tensor.shape)
img = deprocess_image(tensor)
display_images(style_image=img)

Output hidden; open in https://colab.research.google.com to view.

# Defining the Training Configuration

To get everything set up, we first need to build the VGG model, then define a new, more granular, model which contains information (input/output) for each of the layers. We'll use this to extract features from different layers when computing the loss for the image combination.

To combine the 2 images, we'll repeat 2 steps for several iterations: calculate the loss and gradients of the current combined image, then use the optimizer to apply the gradients and modify the combined image towards our desired output.

In [ ]:
#@title # Setting up the New Model

model = build_model()

# Set up a model to extract features, given input, for each layer in the network
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
feature_extractor = Model(inputs=model.inputs, outputs=outputs_dict)


Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [ ]:
#@title # Define the Loss Function

#@markdown We need to define a custom loss function that achieves the desired
#@markdown combination effect of the 2 input images. In the combination image,
#@markdown the content is given by the feature values in the intermediate layers.
#@markdown The style, however, is given by means and correlations of different
#@markdown feature maps. This can be calculated with a Gram matrix, given the
#@markdown features in a layer.

#@markdown This is the most important cell in the notebook. Please go through each defined function
#@markdown and look up each of the TensorFlow functions that are used (in the TensorFlow
#@markdown manual web pages). Make sure you understand each of the functions, including:
#@markdown `transpose`, `reshape`, `matmul`, `reduce_sum`, `concat`, `zeros` and `GradientTape`.

#@markdown Remember that `feature_extractor` was defined above.
#@markdown Check the definition for the gram matrix against the lecture notes.

#@title Loss function

# Calculate the Gram matrix for given input
def gram_matrix(input_tensor):
  input_tensor = tf.transpose(input_tensor, (2, 0, 1))  # Transpose
  features = tf.reshape(input_tensor, (tf.shape(input_tensor)[0], -1))  # Flatten layer
  gram_matrix = tf.matmul(features, tf.transpose(features))
  return gram_matrix

# The style loss is the mean square error between the Gram matrix of the style
# image features and the Gram matrix of the combination image features,
# divided by 4 in original paper (see link at the end of the notebook)
def style_loss(style, combination):
  S = gram_matrix(style)
  C = gram_matrix(combination)
  return tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels ** 2) * (img_size ** 2))

# The content loss is the mean square error between the combination and the content image features
def content_loss(content, combination):
    return tf.reduce_sum(tf.square(combination - content))

# Calculate the loss function given a content image, a style image and the combination result
def loss_function(combination_image, content_image, style_image, content_layers, style_layers):

    # Combine all the images in the same tensor
    input_tensor = tf.concat([content_image, style_image, combination_image], axis=0)

    # Get the features in all the layers for the three images
    features = feature_extractor(input_tensor)

    # Initialise the loss
    loss = tf.zeros(shape=())

    # Extract the content layers and calculate the content loss
    for layer_name in content_layers:
        layer_features = features[layer_name]
        content_image_features = layer_features[0, :, :, :]  # Content image is at position 0
        combination_image_features = layer_features[2, :, :, :]  # Combination image is at position 2
        loss += content_weight * content_loss(content_image_features, combination_image_features)

    # Extract the style layers and calculate the style loss
    for layer_name in style_layers:
        layer_features = features[layer_name]
        style_image_features = layer_features[1, :, :, :]  # Style image is at position 1
        combination_image_features = layer_features[2, :, :, :]
        loss += style_weight * style_loss(style_image_features, combination_image_features)

    return loss

# This brings together the calculation of loss and that of gradients
def compute_loss_and_grads(content_image, style_image, combination_image, content_layers, style_layers):
    with tf.GradientTape() as tape:
        loss = loss_function(combination_image, content_image, style_image, content_layers, style_layers)
        grads = tape.gradient(loss, combination_image)
    return loss, grads

In [ ]:
#@title # TODO 10 - Setting the Weights for the Loss Function

#@markdown Have a look at the style and content layers that we've chosen and see where
#@markdown they are in the output model description from the cell above.

#@markdown Note that we need to compile the optimizer, which is done at the bottom of the cell.
# Total loss is a weighted sum of content and style loss
content_weight = 2.5e-8  #@param
style_weight = 1.2e-6  #@param default is 1e-6

# Set which layers of the network should be taken into consideration when calculating content and style loss
style_layers = [
    "block1_conv1",
    "block2_conv1",
    "block3_conv1",
    "block4_conv1",
    "block5_conv1",
]
content_layers = ["block5_conv2"]

content_weight /= len(content_layers)
style_weight /= len(style_layers)


In [ ]:
#@title # TODO 11 - Define and compile the Optimizer

#@markdown Using the `#@param` keyword, add in three GUI fields to allow the user to define
#@markdown three variables: `initial_learning_rate`, `decay_steps` and `decay_rate`.
#@markdown The default values for these should be `100.0`, `100`, and `0.96` respectively.
#@markdown Code at the bottom of the cell uses these values to define the optimizer.

#@markdown Also, look up the SGD optimizer [here](https://keras.io/api/optimizers/sgd/) and
#@markdown check out the other parameters which could be specified.

#@markdown Note that we have to compile the optimizer, as this does not happen
#@markdown when the object is instantiated.

# TODO 11.1
# Parameters for training
initial_learning_rate = 100.0 # @param {type:"number"}
decay_steps  = 100.0 # @param {type:"number"}
decay_rate = 0.96 # @param {type:"number"}

optimizer = SGD(tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=initial_learning_rate, decay_steps=decay_steps, decay_rate=decay_rate))

# We can now compile (i.e., set up) the optimizer.
model.compile(optimizer, loss_function)

In [ ]:
#@title # TODO 12 - Saving Intermediate Models and Images
#@markdown In the code in this cell, add in some default values for whether to
#@markdown save the image and/or the model.


from google.colab import drive
drive.mount('/content/gdrive')

# Directory where the checkpoints will be saved
path = '/content/gdrive/My Drive/Work/Colab/StyleTransfer/' #@param{type: 'string'}

# TODO 12.1 - Add in default values to this function so that the model isn't saved by default, but the image is.
def save(epoch, img_tensor, save_image, save_model):
  model_name = path + "model_" + str(epoch) + '.tf'
  image_name = path + "img_" + str(epoch) + '.png'

  # Save image
  if save_image:
      img = deprocess_image(img_tensor)
      keras.utils.save_img(image_name, img)

  # Save model
  if save_model:
      tf.saved_model.save(model, model_name)

# This ensures that the path to saving directory exists (so you don't have to create it manually)
os.makedirs(path, exist_ok=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Generating Stylized Images in an Optimization Loop

The goal here is not to train the network, but to first, calculate the loss and gradients for our combination image, and second, apply the gradients to the combination image and transform it towards our desired output.

At each iteration of these 2 steps (or every X iterations if running for longer periods of time) we can display the current state of the result with functions defined earlier, and/or save the images generated and models.

The initial input to the network is going to be 3 tensors: the content image (preprocessed), the syle image (preprocessed) and the combination image (a tensorflow variable, initialised to the values of the content image to speed up the process).

**Implement** this optimization process, following these steps:

1. Create a for or while loop, which should stop after the given number of **iterations**.
2. Within this loop, add the **first step** of the iteration, using functions defined above: calculate the loss and the gradients, given the 3 input tensors.
3. Within the for loop, add the **second step** of the iteration, using the optimizer's `apply_gradients` function. The input to this function is a list of size 1 of tuples containing the gradients and the combination image tensor (which needs to be modified appropriately).
4. **Print** out the current iteration number and loss.
5. Use the function defined earlier to **display** the 3 images (content, style, deprocessed combination image) every 10 or so iterations (this is quite an expensive operation, so it'd take longer if you did it at each iteration).
6. Optionally, **save** the current combination image and/or model using the saving function above.

With everything implemented, run this cell and watch the image transform! (If you want to work from the solutions, feel free to check out the answer below). Don't forget to click the images to see them in detail (smaller image sizes don't usually show the fine texture detail in the stylised image).

Try different style layer choices, as well as weightings for the two loss function parts. And also try different content and style images to see if you can get good results!

In [ ]:
#@title # TODO 13 - Implement the Optimisation loop

# Note that the optimizer has to be redefined on each run (not sure why!)
optimizer = SGD(tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=initial_learning_rate, decay_steps=decay_steps, decay_rate=decay_rate))

iterations = 100  #@param{type: 'number'}

content_image_tensor = preprocess_image(content_image_path)
style_image_tensor = preprocess_image(style_image_path)
combination_image_tensor = tf.Variable(preprocess_image(content_image_path))

# TODO 13 Implement the optimisation routine described above
for iteration in range(iterations):
  # Step 1: calculate loss and gradients
  # TODO 13.1 - Add code here
  loss, grads = compute_loss_and_grads(content_image_tensor, style_image_tensor, combination_image_tensor, content_layers, style_layers)

  # Step 2: apply gradients
  # TODO 13.2 - Add code here
  optimizer.apply_gradients([(grads, combination_image_tensor)])

  # Step 3: Display images every 50 iterations and at the end
  if iteration % 50 == 0 or (iteration == iterations - 1):
    # Display the three images and print the loss
    # TODO 13.3 - Add code here
    combination_image = deprocess_image(combination_image_tensor)
    display_images(content_image, style_image, combination_image)
    print("Loss:", float(loss))
    # NOTE: `content_image` and `style_images` were defined previously

    # Save image and/or model
    # TODO 13.4 - Add code here
    # Copied from solutions as I was lazy...
    save(iteration, combination_image_tensor, save_image=True, save_model=False)

Output hidden; open in https://colab.research.google.com to view.

# Different Approaches

There are other models set up for achieving similar results very quickly and without the need to define any other elements. We can simply load the `tensorflow_hub` package, choose one of the models available and pass into it a content and a style image. This will output an array containing one image, which we can display by turning the resulting tensor back into a Numpy array.

Try this out and check the differences in speed / quality of the output.

In [ ]:
#@title # TODO 14 - TensorFlow Hub

#@markdown Firstly, check out the [TensorFlow Hub](https://www.tensorflow.org/hub)
#@markdown where you can download lots of useful pre-trained models, including ones
#@markdown for style transfer and other effects, like cartoonization.

#@markdown Then run this cell and see the results. Try changing the style and content
#@markdown images using the cells above, and see what you get.

import tensorflow_hub as hub
hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

# We need to scale the image data to 256x256 pixels
# Using the numpy arrays representing our images, with 1 batch dimension added, and normalized
c_img = np.expand_dims(content_image_np, axis=0)/255
s_img = np.expand_dims(style_image_np, axis=0)/255

stylized_images = hub_model(tf.constant(c_img), tf.constant(s_img))[0]
print(stylized_images.shape)
display_images(c_img[0], s_img[0], stylized_images[0])

Output hidden; open in https://colab.research.google.com to view.

# To Explore Further

Here are some more style transfer apps, videos and repositories to check out and play around with!

1. [Arbitrary Style Transfer in the Browser](https://reiinakano.com/arbitrary-image-stylization-tfjs/) is a browser application which allows to select from a range of content and style images to produce a combined image. It also includes some customization from the output, with a choice of different networks as well.

2. [Art Done Quick Style Transfer](http://www.artdonequick.com/). Check out a video of how I (Simon Colton) added style transfer to the Art Done Quick casual creator app.

3. [Magenta Style Transfer](https://magenta.tensorflow.org/blog/2018/12/20/style-transfer-js/). The Magenta team from Google have an in-browser application which works nicely. There are also technical details of how it operates.

4. [AI Painter](https://www.instapainting.com/ai-painter) utilises Style Transfer to customize a picture uploaded by the user into a painting of their liking, with the option of then hiring a painter which would turn the digital output into an actual painting. This is now a commercial site, but check it out to see how people are making money via style transfer.

5. [Ostagram](https://www.ostagram.me/). Go here to see some amazingly good (curated/cherry picked) style transfer examples, and some pretty bad ones!


# Solutions

In [1]:
#@markdown Only look at these if you haven't worked out the answer for yourselves.
#@markdown Also, please ask for help - that's what we're here for!

# VGG is named after the (v)isual (g)eometry (g)roup at the University of Oxford
# https://www.robots.ox.ac.uk/~vgg/
# 16 and 19 refer to the number of convolution layers in the models.

# TODO 4.1
# This is the code that can be copied from what is already there.

# if style_transfer_image is not None:
#    cells[2].imshow(style_transfer_image)
#    cells[2].set_title("Combination", fontsize=30)

# TODO 5.1
# content_image_path = get_file(fname = "content.jpg", origin = content_image_url) # TODO
# style_image_path = get_file(fname = "style.jpg", origin = style_image_url) # TODO

# TODO 5.2
# content_image = keras.utils.load_img(content_image_path)
# style_image = keras.utils.load_img(style_image_path)

# TODO 8.1 - what does the [:, :, 0] notation below mean?
# Answer: it points to the collection of all (r)ed pixel values

# TODO 8.2 - what does the [:, :, ::-1] notation mean?
# Answer: it reverses BGR pixels to RGB

# TODO 8.3 - what does the clip function do?
# Answer: it limits the range of the values (in this case to be between 0 and 255)

# TODO 9.1
# tensor = preprocess_image(style_image_path)
# print(tensor.shape)
# img = deprocess_image(tensor)
# display_images(None, img, None)

# TODO 11.1
# initial_learning_rate = 100.0 #@param
# decay_steps = 100 #@param
# decay_rate = 0.96 #@param

# TODO 12.1
# The function definition needs to change to:
# def save(epoch, img_tensor, save_image=True, save_model=False):

# TODO 13.1
# loss, grads = compute_loss_and_grads(content_image_tensor, style_image_tensor, combination_image_tensor, content_layers, style_layers)

# TODO 13.2
# optimizer.apply_gradients([(grads, combination_image_tensor)])

# TODO 13.3
# print(f"Iteration {iteration + 1} loss={loss:.3f}")
# if iteration % 10 == 0:
#    display_images(content_image, style_image, deprocess_image(combination_image_tensor))

# TODO 13.4
# save(iteration, combination_image_tensor, save_image=True, save_model=False)